In [35]:
import pylab
import imageio
filename = 'arson.mp4'
vid = imageio.get_reader(filename,  'ffmpeg')
frames=[]

for i, im in enumerate(vid):
    frames.append(im)
frames = torch.from_numpy(np.array(frames, dtype ='float'))

In [36]:
frames.shape

torch.Size([4187, 240, 320, 3])

In [37]:
nums = [2000, 2100]
for num in nums:
    image = vid.get_data(num)
    fig = pylab.figure()
    fig.suptitle('image #{}'.format(num), fontsize=20)
    pylab.imshow(image)
pylab.show()

In [41]:
pylab.imshow(frames[1000]/255)
pylab.show()

In [42]:
frames[1000].mean()

tensor(66.13205, dtype=torch.float64)

In [43]:
import torch
import numpy as np
yolo = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True)


Using cache found in C:\Users\derek/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.10 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)

Fusing layers... 


requirements: C:\Users\derek\.cache\torch\hub\requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [44]:
frames = torch.from_numpy(np.array(frames, dtype='float32'))
frames_100 = frames[2000:2100,:,:,:]
print(frames_100.shape)

torch.Size([100, 240, 320, 3])


In [46]:
frames_100[99].mean()

tensor(77.50818)

In [47]:
torch.cuda.is_available()
frames = frames.cpu()
print(frames.device)

cpu


In [48]:
input = [its.cpu().detach().numpy() * 255 for its in frames]
print(input[0].max())

510.0


In [49]:
import pandas as pd

In [51]:
outcome = []
frames_100 = torch.permute(frames_100, (0, 2, 3, 1))
predictions = yolo([its.cpu().detach().numpy() for its in frames_100])

for i in range(frames_100.shape[0]):
        single_result = predictions.pandas().xyxy[i]
        pred_class = single_result["name"].tolist()
        pred_score = single_result["confidence"].tolist()
        pred_boxes = single_result[["xmin", "ymin", "xmax", "ymax"]].apply(
            lambda x: list(x), axis=1
        )

        outcome.append(
                {
                    "labels": pred_class,
                    "bboxes": pred_boxes,
                    "scores": pred_score,
                },
            )
result = pd.DataFrame(
            outcome,
            columns=[
                "labels",
                "bboxes",
                "scores",
            ],
        )

In [53]:
result["labels"]
[3, 1, 1, ]

0     [car, person, skateboard, car, car]
1                 [car, person, car, car]
2                         [car, car, car]
3                 [car, person, car, car]
4                         [car, car, car]
                     ...                 
95                                  [car]
96                                  [car]
97                                  [car]
98                                  [car]
99                                  [car]
Name: labels, Length: 100, dtype: object

In [54]:
result["bboxes"]

0     0    [167.02383422851562, 82.59362030029297, 1...
1     0    [167.0209197998047, 82.59158325195312, 18...
2     0    [167.02590942382812, 82.59093475341797, 1...
3     0    [167.00804138183594, 82.5923080444336, 18...
4     0    [167.0108642578125, 82.57781982421875, 18...
                            ...                        
95    0    [100.26240539550781, 99.43904113769531, 1...
96    0    [100.2587661743164, 99.4135513305664, 135...
97    0    [100.26038360595703, 99.42559814453125, 1...
98    0    [100.27104187011719, 99.4421615600586, 13...
99    0    [100.25949096679688, 99.44524383544922, 1...
Name: bboxes, Length: 100, dtype: object

In [55]:
result["scores"]

0     [0.6064822673797607, 0.5089840292930603, 0.265...
1     [0.6099070906639099, 0.2898620367050171, 0.260...
2     [0.6088065505027771, 0.2546219825744629, 0.250...
3     [0.6069881916046143, 0.32793930172920227, 0.26...
4     [0.6033501029014587, 0.26143956184387207, 0.26...
                            ...                        
95                                 [0.5185160040855408]
96                                 [0.5155003666877747]
97                                  [0.511539876461029]
98                                 [0.5100327730178833]
99                                 [0.5057608485221863]
Name: scores, Length: 100, dtype: object

In [23]:
import cv2
from pprint import pprint
from matplotlib import pyplot as plt

def annotate_video(detections, input_video_path, output_video_path):
    color1=(207, 248, 64)
    color2=(255, 49, 49)
    thickness=4

    vcap = cv2.VideoCapture(input_video_path)
    width = int(vcap.get(3))
    height = int(vcap.get(4))
    fps = vcap.get(5)
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') #codec
    video=cv2.VideoWriter(output_video_path, fourcc, fps, (width,height))

    frame_id = 0
    # Capture frame-by-frame
    # ret = 1 if the video is captured; frame is the image
    ret, frame = vcap.read() 

    while ret:
        df = detections
        df = df[['bboxes', 'labels']][df.index == frame_id]
        if df.size:
            dfLst = df.values.tolist()
            for bbox, label in zip(dfLst[0][0], dfLst[0][1]):
                x1, y1, x2, y2 = bbox
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                # object bbox
                frame=cv2.rectangle(frame, (x1, y1), (x2, y2), color1, thickness) 
                # object label
                cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color1, thickness) 
                # frame label
                cv2.putText(frame, 'Frame ID: ' + str(frame_id), (700, 500), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color2, thickness) 
            video.write(frame)

            # Stop after twenty frames (id < 20 in previous query)
            if frame_id == 4000:
                break

            # Show every fifth frame
            if frame_id % 5 == 0:
                plt.imshow(frame)
                plt.show()

        
        frame_id+=1
        ret, frame = vcap.read()

    video.release()
    vcap.release()

In [24]:
from ipywidgets import Video, Image
input_path = 'arson.mp4'
output_path = 'output.mp4'
annotate_video(result, input_path, output_path)
Video.from_file(output_path)

Video(value=b'\x00\x00\x00\x1cftypisom\x00\x00\x02\x00isomiso2mp41\x00\x00\x00\x08free\x00\x01\xbd\x18...')